# Recurrent Neural Network

<font color='steelblue'>

<font size = 5>
<b>Recurrent Neural Networks Stock Trend Prediction</b>
  
</font>

<font color = 'grey'>
<font size = 4>
<br>
    
- Going to look at the stock price from `2005 to 2021`. Then predict the trend for
2022, the test dataset has the data from the month of `January-October 2022`. Will compare to this test set data and see how good was our trend prediction.<br>
    
**Following examples are included in the processing:**
    

- `Load` training dataset
- `Scale` the data using `MinMaxScaler`
- `Create 60 row buckets` - approximately 3 months
- `Use first 60 rows` are initial data (time t0)
- `Instantiate a RNN model` - set appropriate layers
- `Train` the model
- `Make predictions` using test dataset
- `Plot` trend **prediction v/s actual 2022** data

</font>
</font>
    
<font color = 'tomato'>
    
### NOT GOING TO PREDICT THE STOCK PRICE BUT PREDICT THE TREND - *UP or DOWN or FLAT*
</font>

In [ ]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.layers import LSTM, GRU
from sklearn.metrics import mean_squared_error
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.preprocessing import MinMaxScaler

## Load Training Data <br>
<font size = 3>   
Only load the training dataset, will use the test dataset for comparing the trend that is predicted
</font>

In [ ]:
dataset_train = pd.read_csv('../datasets/Google-Train.csv')

In [ ]:
dataset_train.head()

In [ ]:
dataset_train.describe()

In [ ]:
num_rows = dataset_train.shape[0]
print("number of rows: {}".format(num_rows))

In [ ]:
dataset_train.info()

<span style="font-family:Comic sans MS; font-size:1.4em;">   
Only use the open price for the stock, do not need date, high, low, close values
</span>

In [ ]:
training_set = dataset_train.iloc[:, 1:2].values
training_set[:2]

## Feature Scaling <br>

<font size = 3>   
    
Whenever we are going to use `sigmoid function in the RNN`, it is `recommeneded to use the MinMaxScaler` compared to the standarization scaler.
</font>

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# want to scale between 0 and 1
sc = MinMaxScaler(feature_range = (0, 1))

In [ ]:
# Keep the original training set, hence create new variable
training_set_scaled = sc.fit_transform(training_set)

In [ ]:
training_set_scaled[:2]

## Creating Timestamp Buckets <br>

<font size = 3>
    
- Creating a data structure with `60 timesteps and 1 output`
- `60 timesteps` means at any time t, RNN is going to look at the `stock price of (t - 60) days`
- And based on trend for this period, it will try to `predict the next output`
- So `60 timesteps` is from which our `RNN is going to learn` and understand some correlations and trends and based on this understanding it  will try to `predict the output the stock price at time (t + 1)`
- Using `1 timestep caused overfitting` and the model was not learning anything. `20 timesteps was not able to capture some trends`, then 30 and 40, the `best number of timesteps was 60`
- `60 timesteps corresponds to 3 months` - every month has 20 financial days. We will have 60 timesteps and 1 output - stock price at (t+1)
    
</font>
    </ul>

In [ ]:
# For each finnancial day - X_train will contain the 60 previous days.
# For y_train will be output or the prediction
X_train = []
y_train = []

In [ ]:
# Since we want the previous 60 stock prices, we have to start at index 60 in 
# the training set i.e. 60th financial day of 2021.
for i in range(60, num_rows):
    # top 60 (0 - 59) will be in the first row, and 60-119 in next ....
    # will have num_rows for X_train. Each row will have 60 columns.
    X_train.append(training_set_scaled[i-60:i, 0])
    # 60th will be in the first one, 120th in next. So will have 1198 rows and
    # with 1 column
    y_train.append(training_set_scaled[i, 0])

In [ ]:
print("Number of items in list:", len(X_train))

In [ ]:
# Convert the list to numpy array as required by RNN
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
pd.DataFrame(X_train).head()

## Reshaping the training data<br>

<font size = 3>
    
- Reshaping means adding more dimenonality to the dataset. This dimension we are adding is `a unit - number of predictors` we want to use to predict. Our price predictors are indicators. Here we are only using the open stock price which is our only indicator. You can have many indicators - close, volume, high, low, etc. By adding this additional  indicator, it will have predict the trend in the stock price. the `input shape has to be 3D as required by the RNN`.


- Anytime need to add dimenion in numpy array, need to use the reshape function to do it,
    `reshape requires 2 arguments` - what we want to reshape - `X_train and new shape our numpy array` to have. We are converting from `2D array to 3D array`. The order in which the new dimenions are to be added go to  `'keras documentation'` and look for Recurrent layers in the layers section.<br>
    
<ul>
        
<li> <b>Check the input_shape:</b> <i>3D tensor with shape - requires  (batch_size, timesteps, input_dim)</i> </li>
<li><b>batch_size:</b> total number of rows we have in the training set (1198). </li>
    <li><b>timesteps:</b> total number of columns for each row. </li>
    <li><b>input_dim:</b> is the new dimension we are adding - the indicator or predictor </li>
    <li><b>X_train.shape[0]</b> - will be number of rows, X_train.shape[1] will be number of columns</li>
</ul>
    

</font>

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
X_train.shape

## Building the RNN

In [ ]:
# Initializing the RNN

# regression is about predicting continous value and classificaiton is about
# predicting a category or class.
tf.random.set_seed(2345)
regressor = Sequential()

## Adding the first LSTM layer and some Dropout regularization:<br>
<ul>
<font size = 3>
    <li><b>units</b> - number of LSTM cells/memory units/neurons for the LSTM layer. Even if we are going to stack the LSTM layers, we want our model to have high dimensionality. To do this we can add large number of neurons in each layer. Since predicting the stock price is complex, having a large number of neurons in each layer will help. If we had taken 3 or 5 units in each layer then the model will not be able to capture the upward and the downward trend of the stock price.</li> 
    <li><b>return_sequences</b> - since we are building a stacked LSTM, we set it to true, in the next steps we are going to add additional LSTM layers.</li>
    <li><b>input_shape</b> - the input shape we created in the Reshaping step, a 3D array - containing the rows, columns and the indicators. In here we have to only add the last 2 dimemsions (number columns and the indicator), the first dimension will be automatically taken into account.</li> 
    
</font>
    </ul>

In [ ]:
regressor.add(LSTM(units = 50, return_sequences = True, 
                   input_shape = (X_train.shape[1], 1)))

In [ ]:
# add dropout regularization to avoid over fitting. (20 %)
#regressor.add(Dropout(0.2))

## Adding the second LSTM layer and some Dropout regularization:<br>

<font size = 3>
Adding a second LSTM layer and some Dropout regularization want to keep return_sequences to true because we are going to add more layers no need to add input_shape because we have already defined in step 1.
</font>

In [ ]:
# add LSTM layer and then drop 20% to prevent overfitting

regressor.add(LSTM(units = 50, return_sequences = True))
#regressor.add(Dropout(0.2))

## Adding the third LSTM layer and some Dropout regularization:<br>

<font size = 3>
    
Want to keep return_sequences to true because we are going to add more layers
</font>

In [ ]:
# add LSTM Layer and then drop 20% to prevent overfitting

regressor.add(LSTM(units = 50, return_sequences = True))
#regressor.add(Dropout(0.2))

## Adding the fourth and final LSTM layer and some Dropout regularization:<br>

<font size = 3>
Final layer so use default for return_sequence (set to false)
</font>

In [ ]:
# Adding a fourth LSTM layer and some Dropout regularization
# final layer so use default return_sequence (set to false)
regressor.add(LSTM(units = 50))
#regressor.add(Dropout(0.2))

## Adding output layer<br>

<font size = 3>
Have on predictor hence units = 1. Making full connection to the previous LSTM layer, use the Dense class.
</font>

In [ ]:
regressor.add(Dense(units = 1))

In [ ]:
# Get regressor summary
regressor.summary()

In [ ]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

## Train the RNN
<ul>
<font size = 3>
    
**First two arguements are X_train and y_train**
    <li><b>epochs</b> - he number of forward and backward propagations to be done on the data. With 25 there was no convergence of loss then tried 50 - still not some convergence, then tried 100 were observed some convergence.</li> 
    <li><b>batch_size</b> - Every x times before the back propagation happens of the weights</li>
<b>The batch_size and the epochs could be set as variables rather than hard coding them</b>
</font>
    </ul>

In [ ]:
%%time
tf.random.set_seed(2345)
history = regressor.fit(X_train, y_train, epochs = 35, batch_size = 32, verbose = 1)

In [ ]:
loss = history.history['loss']
plt.figure(figsize=(4,3))
epoch = history.epoch
plt.plot(epoch,loss);

<font color='grey'>
    <h2>Observe loss</h2>

<font size = 3><br>
The loss goes keeps decreasing from first epoch to last. Have prevented the over fitting enough so that the loss does not much change in the last 20 ~ 30 epochs.
</font>
</font>

## Making the predictions and visualizing the results

In [ ]:
# the csv has the stock price for month of january 2022
# Getting the real stock price of 2022
dataset_test = pd.read_csv('../datasets/Google-Test.csv')
test_rows = dataset_test.shape[0]
print("number of rows in test set: {}".format(test_rows))
real_stock_price = dataset_test.iloc[:, 1:2].values

## Predicting stock price for 2022
<ul>
<font size = 3>
Regressor was built using the 60 previous stock prices, to predict the 2022 stock price we will need the 60 prior days stock prices. In order to get these we will need both the training set and the test set. Some data will be from October, November and December 2021<br><br>
We will have to do concatenation of the training set and the test set. We cannot do this since the training set is scaled whereas the test (real price) is not scaled. We do not want to scale the test set because we do not want to loose the actual values in the test set. To do this concatenation, we will use the dataset_train and dataset_test both of which have the real stock prices.
</font>

In [ ]:
# 2 Arguments - 2 datasets that need to be concatenated and the axis along 
# which we want to concatenate - want to concatenate rows or columns.
# axis = 0  means rows
# dataset_train['Open'] - we are getting column named Open from the dataset.
dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']), 
                          axis = 0)

In [ ]:
dataset_total.head()

In [ ]:
print(type(dataset_total))

In [ ]:
# We are trying to get the prices using the last 3 months dec, nov, oct.
# get to jan 3rd 2022 - len(dataset_total) - len(dataset_test)
# to get the lower bound - subract 60 from jan 1st 2022. 
# the upper bound is the rest - the last index of the whole dataset
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
print(inputs.shape)
inputs

In [ ]:
# inputs is now just a numpy array of one dimension (x, ). We need to 
# transform it to have (x,1) - have one column -reshape it with values -1, 1.
inputs = inputs.reshape(-1,1)
inputs.shape

In [ ]:
# now standardize the inputs because our regressor was scaled.
# sc was defined earlier
inputs = sc.transform(inputs)

In [ ]:
# now transform it into the 3D format which we passed into the regressor.
X_test = []
# we have test_row entries
for i in range(60, test_rows+60):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)

In [ ]:
# create the 3D structure required
#
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
X_test.shape

In [ ]:
%%time
# predict the stock price
predicted_stock_price = regressor.predict(X_test)

In [ ]:
# since the prediction is scaled - need to inverse transform it using
# the scaler before it can be plotted
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

In [ ]:
predicted_stock_price[:2]

In [ ]:
# real stock prices from January 2022 v/s our prediction trend
plt.figure(figsize = (12, 8))
plt.plot(real_stock_price, color = 'red', label = 'Real Google Stock Price')
plt.plot(predicted_stock_price, color = 'blue', 
         label = 'Predicted Google Stock Price Trend')
plt.title('Google Stock Price Prediction (2022)')
plt.xlabel('Time')
plt.ylabel('Google Stock Price')
plt.legend()
plt.show()